Скрэпим исполнитель + id исполнителя + id есни + название + продолжительность + ссылка

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import random
import os
import urllib.parse

def read_artists_from_csv(filename):
    artists = []
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if 'Rank' in row and row['Rank']:
                    artists.append(row['Rank'])
        print(f"Прочитано артистов из CSV: {len(artists)}")
        return artists
    except Exception as e:
        print(f"Ошибка чтения CSV: {e}")
        return []

def search_artist(artist_name):
    search_url = f"https://rus.hitmotop.com/search?q={urllib.parse.quote(artist_name)}"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }
    
    try:
        print(f"Ищем артиста: {artist_name}")
        response = requests.get(search_url, headers=headers)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        performers_section = None
        for h2 in soup.find_all('h2'):
            if 'Исполнители' in h2.get_text():
                performers_section = h2.find_next('ul')
                break
        
        if performers_section:
            first_artist_link = performers_section.find('a', href=lambda x: x and x.startswith('/artist/'))
            if first_artist_link:
                artist_href = first_artist_link.get('href')
                artist_id = artist_href.split('/artist/')[-1]
                print(f"Найден артист: {artist_name} -> ID: {artist_id}")
                return artist_id
        
        print(f"Артист не найден: {artist_name}")
        return None
        
    except Exception as e:
        print(f"Ошибка поиска артиста {artist_name}: {e}")
        return None

def get_artist_tracks(artist_id, artist_name, max_pages=1):
    """Получаем треки артиста"""
    base_url = f"https://rus.hitmotop.com/artist/{artist_id}"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }
    
    all_tracks = []
    
    for page in range(1, max_pages + 1):
        try:
            if page == 1:
                url = base_url
            else:
                start_num = (page - 1) * 48
                url = f"{base_url}/start/{start_num}"
            
            print(f"Страница {page}: {url}")
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.content, 'html.parser')
            
            track_elements = soup.select('a[href^="/song/"]')
            
            for track_element in track_elements:
                href = track_element.get('href')
                if href and href.startswith('/song/'):
                    track_id = href.split('/song/')[-1]
                    
                    track_info = get_track_info(track_id, artist_name)
                    if track_info:
                        track_info['artist_id'] = artist_id
                        all_tracks.append(track_info)
            
            print(f"Страница {page}: найдено {len(track_elements)} треков")
            
            if not soup.select('a[href*="/start/"]'):
                break
                
            time.sleep(random.uniform(1, 2))
            
        except Exception as e:
            print(f"Ошибка на странице {page}: {e}")
            continue
    
    return all_tracks

def get_track_info(track_id, artist_name):
    track_url = f"https://rus.hitmotop.com/song/{track_id}"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }
    
    try:
        response = requests.get(track_url, headers=headers)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        title_element = soup.find('h1', class_='p-track-title')
        if not title_element:
            return None
        
        track_text = title_element.get_text(strip=True)
        
        if ' - ' in track_text:
            title = track_text.split(' - ', 1)[1]
        else:
            title = track_text
        
        duration_element = soup.select_one('.track__fulltime')
        duration = duration_element.get_text(strip=True) if duration_element else 'N/A'
        
        track_info = {
            'artist': artist_name,
            'artist_id': '',
            'track_id': track_id,
            'title': title,
            'duration': duration,
            'url': track_url
        }
        
        print(f"{artist_name} - {title} | {duration}")
        return track_info
        
    except Exception as e:
        print(f"Ошибка получения информации о треке {track_id}: {e}")
        return None

def save_tracks_to_csv(tracks, filename):
    try:
        with open(filename, 'w', newline='', encoding='utf-8-sig') as file:
            writer = csv.writer(file)
            writer.writerow(['Исполнитель', 'ID исполнителя', 'ID песни', 'Название песни', 'Продолжительность', 'Ссылка'])
            
            for track in tracks:
                writer.writerow([
                    track['artist'],
                    track['artist_id'],
                    track['track_id'],
                    track['title'],
                    track['duration'],
                    track['url']
                ])
        
        print(f"Сохранено {len(tracks)} треков в файл: {filename}")
        return True
        
    except Exception as e:
        print(f"Ошибка сохранения csv: {e}")
        return False

def main():
    print("=== HITMOTOP ARTIST TRACKS PARSER ===")
    
    csv_filename = 'spotify_artists.csv'
    artists = read_artists_from_csv(csv_filename)
    
    if not artists:
        print("Не удалось прочитать артистов из csv-файла")
        return
    
    #options!!!!!!!!!!!!!!!!!!! важно
    start_index = 0
    end_index = 300
    max_pages_per_artist = 1
    
    print(f"\n Парсим артистов с {start_index} по {end_index} индекс")
    
    all_tracks = []
    
    for i in range(start_index, min(end_index, len(artists))):
        artist_name = artists[i]
        print(f"\n{'='*50}")
        print(f"Обрабатываем артиста {i+1}/{min(end_index, len(artists))}: {artist_name}")
        
        artist_id = search_artist(artist_name)
        if not artist_id:
            continue
        
        artist_tracks = get_artist_tracks(artist_id, artist_name, max_pages_per_artist)
        all_tracks.extend(artist_tracks)
        
        print(f"Артист {artist_name}: собрано {len(artist_tracks)} треков")
        
        time.sleep(random.uniform(2, 4))
    
    if all_tracks:
        output_filename = 'artist_tracks.csv'
        save_tracks_to_csv(all_tracks, output_filename)
        print(f"\n ВСЕГО СОБРАНО ТРЕКОВ: {len(all_tracks)}")
        print(f"Файл: {output_filename}")
    else:
        print("\n Не удалось собрать ни одного трека")

if __name__ == "__main__":
    main()
    print("\n Нажмите enter для выхода...")
    input()

Дальше Скрепим id трэка, id жанра, жанр, страница в жанре, ссылка на жанр

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import random
import os

def read_track_ids_from_csv(csv_file):
    track_ids = []
    try:
        with open(csv_file, 'r', encoding='utf-8-sig') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if 'ID песни' in row and row['ID песни']:
                    track_ids.append(row['ID песни'])
        print(f"Прочитано ID песен из CSV: {len(track_ids)}")
        return track_ids
    except Exception as e:
        print(f"Ошибка чтения CSV: {e}")
        return []

def find_track_genre(track_id):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }
    
    for genre_id in range(1, 240):
        print(f"🔍 Проверяем жанр {genre_id}/124 для трека {track_id}...")
        
        for page in range(1, 8):
            try:
                if page == 1:
                    url = f"https://rus.hitmotop.com/genre/{genre_id}"
                else:
                    start_num = (page - 1) * 48
                    url = f"https://rus.hitmotop.com/genre/{genre_id}/start/{start_num}"
                
                response = requests.get(url, headers=headers)
                response.raise_for_status()
                
                soup = BeautifulSoup(response.content, 'html.parser')
                
                track_elements = soup.select('a[href^="/song/"]')
                
                for track_element in track_elements:
                    href = track_element.get('href')
                    if href and f"/song/{track_id}" in href:
                        genre_name = get_genre_name(genre_id, headers)
                        print(f"Найден трек {track_id} в жанре {genre_name} (ID: {genre_id})")
                        return {
                            'track_id': track_id,
                            'genre_id': genre_id,
                            'genre_name': genre_name,
                            'found_on_page': page,
                            'genre_url': f"https://rus.hitmotop.com/genre/{genre_id}"
                        }
                
                print(f" Страница {page}: трек не найден")
                
                if not soup.select('a[href*="/start/"]'):
                    break
                    
                time.sleep(random.uniform(0.5, 1))
                
            except Exception as e:
                print(f"Ошибка при проверке жанра {genre_id}, страница {page}: {e}")
                continue
    
    print(f"Трек {track_id} не найден ни в одном жанре")
    return None

def get_genre_name(genre_id, headers):
    try:
        url = f"https://rus.hitmotop.com/genre/{genre_id}"
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        title_element = soup.find('h1') or soup.find('h2')
        if title_element:
            return title_element.get_text(strip=True)
        
        sidebar_genres = soup.select('.sidebar-genres a')
        for genre_link in sidebar_genres:
            href = genre_link.get('href', '')
            if f"/genre/{genre_id}" in href:
                return genre_link.get_text(strip=True)
        
        return f"Жанр_{genre_id}"
        
    except Exception as e:
        print(f"Не удалось получить название жанра {genre_id}: {e}")
        return f"Жанр_{genre_id}"

def save_results_to_csv(results, filename, save_path=None):
    
    if save_path is None:
        save_path = os.path.dirname(os.path.abspath(__file__))
    
    full_filename = os.path.join(save_path, filename)
    
    try:
        with open(full_filename, 'w', newline='', encoding='utf-8-sig') as file:
            writer = csv.writer(file)
            writer.writerow(['ID трека', 'ID жанра', 'Жанр', 'Страница', 'Ссылка на жанр'])
            
            for result in results:
                writer.writerow([
                    result['track_id'],
                    result['genre_id'],
                    result['genre_name'],
                    result['found_on_page'],
                    result['genre_url']
                ])
        
        print(f"Все результаты сохранены в файл: {full_filename}")
        return True
        
    except Exception as e:
        print(f"Ошибка сохранения CSV: {e}")
        return False

def main():
    print("=== HITMOTOP TRACK GENRE FINDER ===")
    
    # настройки
    input_csv = "artist_tracks.csv" 
    output_csv = "tracks_with_genres.csv"
    max_tracks_to_process = 10000
    
    track_ids = read_track_ids_from_csv(input_csv)
    
    if not track_ids:
        print(f"Не удалось прочитать ID треков из файла {input_csv}")
        return
    
    track_ids = track_ids[:max_tracks_to_process]
    print(f"Будет обработано треков: {len(track_ids)}")
    
    all_results = []
    found_count = 0
    not_found_count = 0
    
    for i, track_id in enumerate(track_ids, 1):
        print(f"\n{'='*50}")
        print(f"Обрабатываем трек {i}/{len(track_ids)}: {track_id}")
        
        result = find_track_genre(track_id)
        
        if result:
            all_results.append(result)
            found_count += 1
            print(f"Найден жанр для трека {track_id}")
        else:
            not_found_count += 1
            print(f"Жанр для трека {track_id} не найден")
        
        if i < len(track_ids):
            delay = random.uniform(2, 4)
            print(f"Ждем {delay:.1f} сек перед следующим треком...")
            time.sleep(delay)
    
    if all_results:
        save_results_to_csv(all_results, output_csv)
        
        print(f"\n ИТОГИ ПОИСКА:")
        print(f"Обработано треков: {len(track_ids)}")
        print(f"Найдено жанров: {found_count}")
        print(f"Не найдено: {not_found_count}")
        print(f"Файл с результатами: {output_csv}")
    else:
        print(f"\n Не удалось найти жанры ни для одного трека")

if __name__ == "__main__":
    main()
    print("\n Нажмите enter для выхода...")
    input()